In [1]:
%%time

from core.TrajectoryDF import NumPandasTraj as NumTrajDF
from features.temporal_features import TemporalFeatures as temporal
import pandas as pd
from utilities.conversions import Conversions as con

atlantic = pd.read_csv('./data/atlantic.csv')
atlantic = con.convert_directions_to_degree_lat_lon(atlantic, 'Latitude',"Longitude")
atlantic.head()

CPU times: user 6.87 s, sys: 663 ms, total: 7.54 s
Wall time: 6.62 s


,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,AL011851,UNNAMED,18510625,0,,HU,28.0,-94.8,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,AL011851,UNNAMED,18510625,600,,HU,28.0,-95.4,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,AL011851,UNNAMED,18510625,1200,,HU,28.0,-96.0,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,AL011851,UNNAMED,18510625,1800,,HU,28.1,-96.5,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,AL011851,UNNAMED,18510625,2100,L,HU,28.2,-96.8,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


In [2]:
%%time
# This cell is taken from the PyMove library.
# They are cited in the paper as well.

def convert_to_datetime(row):
        this_date = '{}-{}-{}'.format(str(row['Date'])[0:4], str(row['Date'])[4:6], str(row['Date'])[6:])
        this_time = '{:02d}:{:02d}:00'.format(int(row['Time']/100), int(str(row['Time'])[-2:]))
        return '{} {}'.format(this_date, this_time)
atlantic['DateTime'] = atlantic.apply(convert_to_datetime, axis=1)
atlantic.head()

CPU times: user 1.72 s, sys: 8.43 ms, total: 1.72 s
Wall time: 1.72 s


,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW,DateTime
0,AL011851,UNNAMED,18510625,0,,HU,28.0,-94.8,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,1851-06-25 00:00:00
1,AL011851,UNNAMED,18510625,600,,HU,28.0,-95.4,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,1851-06-25 06:00:00
2,AL011851,UNNAMED,18510625,1200,,HU,28.0,-96.0,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,1851-06-25 12:00:00
3,AL011851,UNNAMED,18510625,1800,,HU,28.1,-96.5,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,1851-06-25 18:00:00
4,AL011851,UNNAMED,18510625,2100,L,HU,28.2,-96.8,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,1851-06-25 21:00:00


In [3]:
%%time

num_atlantic = NumTrajDF(atlantic,
                         latitude='Latitude',
                         longitude='Longitude',
                         datetime='DateTime',
                         traj_id='ID',
                         rest_of_columns=[])
num_atlantic.drop('Date', inplace=True, axis=1)
num_atlantic.drop('Time', inplace=True, axis=1)
num_atlantic.head()


CPU times: user 48 ms, sys: 2 µs, total: 48 ms
Wall time: 46.6 ms


,,Name,Event,Status,lat,lon,Maximum Wind,Minimum Pressure,Low Wind NE,Low Wind SE,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
DateTime,traj_id,,,,,,,,,,,,,,,,,,,
1851-06-25 00:00:00,AL011851,UNNAMED,,HU,28.0,-94.8,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1851-06-25 06:00:00,AL011851,UNNAMED,,HU,28.0,-95.4,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1851-06-25 12:00:00,AL011851,UNNAMED,,HU,28.0,-96.0,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1851-06-25 18:00:00,AL011851,UNNAMED,,HU,28.1,-96.5,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1851-06-25 21:00:00,AL011851,UNNAMED,L,HU,28.2,-96.8,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


In [4]:
%%time

num_atlantic = temporal.create_date_column(num_atlantic)

CPU times: user 1.04 s, sys: 263 ms, total: 1.3 s
Wall time: 1.15 s


In [5]:
%%time
num_atlantic = temporal.create_time_column(num_atlantic)

CPU times: user 1 s, sys: 284 ms, total: 1.29 s
Wall time: 1.23 s


In [6]:
%%time

num_atlantic = temporal.create_day_of_week_column(num_atlantic)

CPU times: user 962 ms, sys: 272 ms, total: 1.23 s
Wall time: 1.25 s


In [7]:
%%time
num_atlantic = temporal.create_weekend_indicator_column(num_atlantic)
num_atlantic.head()

CPU times: user 688 ms, sys: 178 ms, total: 866 ms
Wall time: 1.07 s


,,Name,Event,Status,lat,lon,Maximum Wind,Minimum Pressure,Low Wind NE,Low Wind SE,Low Wind SW,...,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW,Date,Time,Day_Of_Week,Weekend
DateTime,traj_id,,,,,,,,,,,,,,,,,,,,,
1851-06-25 00:00:00,AL011851,UNNAMED,,HU,28.0,-94.8,80,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,1851-06-25,00:00:00,Wednesday,False
1851-06-25 06:00:00,AL011851,UNNAMED,,HU,28.0,-95.4,80,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,1851-06-25,06:00:00,Wednesday,False
1851-06-25 12:00:00,AL011851,UNNAMED,,HU,28.0,-96.0,80,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,1851-06-25,12:00:00,Wednesday,False
1851-06-25 18:00:00,AL011851,UNNAMED,,HU,28.1,-96.5,80,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,1851-06-25,18:00:00,Wednesday,False
1851-06-25 21:00:00,AL011851,UNNAMED,L,HU,28.2,-96.8,80,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,1851-06-25,21:00:00,Wednesday,False


In [9]:
%%time
# Now lets calculate the traj duration of the hurricane with
# AL011851 traj_id.

time_delta = temporal.get_traj_duration(num_atlantic, traj_id='AL091973')
time_delta

CPU times: user 52 ms, sys: 418 µs, total: 52.5 ms
Wall time: 50.1 ms


DateTime   10 days 06:00:00
dtype: timedelta64[ns]